In [3]:
import mysql.connector

# credentials for connecting to the MySQL db
import json
db_creds = json.load(open('hidden/creds.json'))

def getDBCursor(creds):
    """
    Get the cursor for the db to perform transaction.

    Input: creds (dictionary of login credentials)
    Output: MySQL Connection, MySQl cursor object
    """
    cnx = mysql.connector.connect(**creds)
    return cnx, cnx.cursor()

def queryDB(cursor, query):
    """
    Execute a sql query.

    Input: cursor (MySQL cursor object), query (string of sql query)
    Output: dictionary containing the lines of the query
    """
    cursor.execute(query)
    return cursor.fetchall()


def executeQuery(creds, query):
    """
    Executes a query by connecting, querying, and closes the db connection

    Input: creds (db login credentials), query (string of sql query)
    Output: dictionary of query results
    """
    result = None
    try:
        conn, cursor = getDBCursor(creds)
        result = queryDB(cursor, query)
    except Error as e:
        print(e)
    finally:
        conn.close()
        cursor.close()
    return result

In [4]:
col_names = executeQuery(db_creds, "describe Bootcamp.Postal_Codes_Tbl")
data = executeQuery(db_creds, "select * from Bootcamp.Postal_Codes_Tbl limit 1")

In [5]:
x = []
for i in col_names:
    x.append(str(i[0]))
y = []
for i in col_names:
    y.append(str(i[1]))
z = []
for i in data[0]:
    z.append(str(i))

In [6]:
data

[(u'210',
  u'Portsmouth',
  u'New Hampshire',
  u'NH',
  u'Rockingham',
  u'43.0059',
  u'-71.0132')]

In [90]:
f = open('data_info.csv', 'w')

In [91]:
f.write(','.join(x))
f.write('\n')
f.write(','.join(y))
f.write('\n')
f.write(','.join(z))

In [92]:
f.close()

In [94]:
import boto3

In [101]:
s3 = boto3.client('s3')
response = s3.list_buckets()

In [102]:
for bucket in response['Buckets']:
    print bucket['Name']

bootcamp-sadepu-testing-01
bootcamp-testing-01
bootcamp-testing-02
bootcampaugust2017-1a
bootcampaugust2017-1b
bootcampaugust2017-2a
bootcampaugust2017-2b
bootcampaugust2017-3a
bootcampaugust2017-3b
bootcampjanuary2018-1a
bootcampjanuary2018-1b
bootcampjanuary2018-2a
bootcampjanuary2018-2b
bootcampjuly2017-01
bootcampjuly2017-02
bootcampjuly2017-1a
bootcampjuly2017-1b
bootcampjuly2017-2a
bootcampjuly2017-2b
bootcampjuly2017-3a
bootcampjuly2017-3b
cloudtrail-ctvdev
codebuild-nva-jkampf-input-bucket
codebuild-nva-jkampf-output-bucket
ctv-appstore-artifacts
ctv-appstore-images
ctvstreamingmediadrop
ctvstreamingmediastore
ctvstreamingmediathumbs
gitlab.captechlab.com
intern16-01
mobilejumpstartkit
septanav
sitecore-dev-audio


In [106]:
s3.upload_file('data_info.csv','bootcampjanuary2018-1a','data_info.csv')

S3UploadFailedError: Failed to upload data_info.csv to bootcampjanuary2018-1a/data_info.csv: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied

In [107]:
s3.get_bucket_acl(Bucket='bootcampjanuary2018-1a')

ClientError: An error occurred (AccessDenied) when calling the GetBucketAcl operation: Access Denied

In [1]:
import json
json.load(open('hidden/creds.json'))

{u'database': u'Bootcamp',
 u'host': u'captechbootcamp.cat1vmhhjrmh.us-east-1.rds.amazonaws.com',
 u'password': u'123data',
 u'user': u'captecher'}